In [1]:
#Loading yelp dataset
import json
import pandas as pd
users = []
with open(r"C:\Users\shaha\Desktop\IBS\yelp\yelp_academic_dataset_review.json",encoding='utf-8') as fl:
    for i, line in enumerate(fl):
        users.append(json.loads(line))
        if i+1 >= 20000:
            break
rev = pd.DataFrame(users)
rev=rev.drop(columns=['review_id','user_id','business_id','stars','date','useful','funny','cool'])
rev

,text
0,"If you decide to eat here, just be aware it is..."
1,I've taken a lot of spin classes over the year...
2,Family diner. Had the buffet. Eclectic assortm...
3,"Wow! Yummy, different, delicious. Our favo..."
4,Cute interior and owner (?) gave us tour of up...
...,...
19995,Manager is a complete asshole. If you have a ...
19996,"If you're looking for Bingo around St Louis, t..."
19997,Unfortunately we had a bad experience here...w...
19998,Meh. This pizza was basically a deep-dish grea...


In [21]:
import pandas as pd
#This contain 200 data which is labelled initially for training
train_data= pd.read_csv(r"C:\Users\shaha\Desktop\IBS\yelp\train_data_init.csv",index_col=False)
#This contain 87 data which is labelled for testing
test_data=pd.read_csv(r"C:\Users\shaha\Desktop\IBS\yelp\test_data.csv",index_col=False)

In [22]:
#cleaning the text
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")
import nltk
from nltk.corpus import stopwords
my_stopwords = set(stopwords.words('english') + ['super', 'duper', 'place'])
from nltk.tokenize import word_tokenize
import string
def clean(tex):
  tex = tex.replace("&amp;amp;", '').replace("\'",'') 
  tex=tex.str.replace(r'\d+','')
  tex = tex.str.replace('@user', '')
  tex = tex.str.replace("[^a-zA-Z#]", " " )
  tex = tex.str.replace("#", " ")
  tex = tex.str.replace("-", " ")
  tex = tex.str.replace("$", " ")
  tex = tex.apply(lambda x: x.split())
  tex= tex.apply(lambda x: [stemmer.stem(i) for i in x])
  def process(text):
    # Check characters to see if they are in punctuation
    nopunc = set(char for char in list(text) if char not in string.punctuation)
    # Join the characters to form the string.
    nopunc = " ".join(nopunc)
    # remove any stopwords if present
    return [word for word in nopunc.lower().split() if word.lower() not in my_stopwords]
  tex = tex.apply(process)
  return tex

In [23]:
#Active learning
import scipy.sparse as sp
import pandas as pd
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV
from sklearn.ensemble import RandomForestClassifier
def add_data(x,y):
    x=sp.vstack((x,y))
    return x
def add_label(x,y):
    x=np.concatenate((x,y))
    return x
def fit_model(x,y):
    param_grid = {'max_depth': [3, 5, 10],
                 'min_samples_split': [2, 5, 10]}
    #model=SVC(kernel = 'linear',gamma = 'scale', shrinking = False)
    #model=LogisticRegression(class_weight='balanced', random_state=42)
    base_estimator = RandomForestClassifier(bootstrap=True,class_weight=None,criterion='gini',
                                    max_depth=None,max_features='auto',max_leaf_nodes=None,n_estimators=200,
                                    min_samples_leaf=1,min_samples_split=2,min_weight_fraction_leaf=0.0,
                                    verbose=0,warm_start=False ,oob_score = True,n_jobs = 1,random_state=1)
    model = HalvingGridSearchCV(base_estimator, param_grid, cv=5,
                            factor=2, resource='n_estimators',
                            max_resources=30)
    model.fit(x,y)
    return model
from scipy.stats import entropy
import numpy as np
def multi_argmax(values, n_instances=110):
    max_idx = np.argpartition(-values, n_instances-1, axis=0)[:n_instances]
    return max_idx
def entropy_sampling( X,n_instances= 110):
    ent=np.transpose(entropy(np.transpose(X)))
    return multi_argmax(ent, n_instances=n_instances)

In [25]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer,TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [26]:
train_data['text']

0      If you decide to eat here, just be aware it is...
1      I've taken a lot of spin classes over the year...
2      Family diner. Had the buffet. Eclectic assortm...
3      Wow!  Yummy, different,  delicious.   Our favo...
4      Cute interior and owner (?) gave us tour of up...
                             ...                        
193    I paid well over what I was wanting to and wha...
194    Gaylord Opryland: A Country Christmas Show wit...
195    Cute setting in Lafayette Square. Homemade Bre...
196    My iPhone 6 screen cracked and the LCD was dam...
197    I wish I read these reviews before we hired Ma...
Name: text, Length: 198, dtype: object

In [27]:
#cleaning train and test data
train_data_text=clean(train_data['text'])
test_data_text=clean(test_data['text'])

<ipython-input-22-592f685e97cd>:10: FutureWarning: The default value of regex will change from True to False in a future version.
  tex=tex.str.replace(r'\d+','')
<ipython-input-22-592f685e97cd>:12: FutureWarning: The default value of regex will change from True to False in a future version.
  tex = tex.str.replace("[^a-zA-Z#]", " " )
<ipython-input-22-592f685e97cd>:15: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  tex = tex.str.replace("$", " ")


In [28]:
#Building Initial model and checking the accuracy
count_vect = CountVectorizer(stop_words='english')
transformer = TfidfTransformer(norm='l2',sublinear_tf=True)
x_train_counts = count_vect.fit_transform(train_data_text.apply(lambda x: ' '.join(x)))
x_train_tfidf = transformer.fit_transform(x_train_counts)
label=train_data['label'].to_numpy()
model=fit_model(x_train_tfidf,label)
print("length of training data: ", len(train_data))
print("length of testing data: ", len(test_data))
x_test_counts = count_vect.transform(test_data_text.apply(lambda x: ' '.join(x)))
x_test_tfidf = transformer.transform(x_test_counts)
predictions_train = model.predict(x_train_tfidf)
y_test=train_data['label']
print('train accuracy : ',accuracy_score(y_test,predictions_train)*100)
predictions_test = model.predict(x_test_tfidf)
y_test=test_data['label']
print('test accuracy: ',accuracy_score(y_test,predictions_test)*100)


c:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:541: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
c:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:545: RuntimeWarning: invalid value encountered in true_divide
  decision = (predictions[k] /
c:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:541: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
c:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:545: RuntimeWarning: invalid value encountered in true_divide
  decision = (predictions[k] /
c:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:541: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were 

length of training data:  198
length of testing data:  87
train accuracy :  96.46464646464646
test accuracy:  72.41379310344827


For the time being the learned data are added to initial training data and saved as seperate files

In [29]:
pool_data= pd.read_csv(r"C:\Users\shaha\Desktop\IBS\yelp\train_data3.csv",index_col=False)
train_data_text=clean(pool_data['text'])
test_data_text=clean(test_data['text'])
count_vect = CountVectorizer(stop_words='english')
transformer = TfidfTransformer(norm='l2',sublinear_tf=True)
x_train_counts = count_vect.fit_transform(train_data_text.apply(lambda x: ' '.join(x)))
x_train_tfidf = transformer.fit_transform(x_train_counts)
label=pool_data['label'].to_numpy()
model=fit_model(x_train_tfidf,label)
print("length of training data: ", len(pool_data))
print("length of testing data: ", len(test_data))
x_test_counts = count_vect.transform(test_data_text.apply(lambda x: ' '.join(x)))
x_test_tfidf = transformer.transform(x_test_counts)
predictions_train = model.predict(x_train_tfidf)
y_test=pool_data['label']
print('train accuracy : ',accuracy_score(y_test,predictions_train)*100)
predictions_test = model.predict(x_test_tfidf)
y_test=test_data['label']
print('test accuracy: ',accuracy_score(y_test,predictions_test)*100)

<ipython-input-22-592f685e97cd>:10: FutureWarning: The default value of regex will change from True to False in a future version.
  tex=tex.str.replace(r'\d+','')
<ipython-input-22-592f685e97cd>:12: FutureWarning: The default value of regex will change from True to False in a future version.
  tex = tex.str.replace("[^a-zA-Z#]", " " )
<ipython-input-22-592f685e97cd>:15: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  tex = tex.str.replace("$", " ")
c:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:541: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
c:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:545: RuntimeWarning: invalid value encountered in true_divide
  dec

length of training data:  312
length of testing data:  87
train accuracy :  94.87179487179486
test accuracy:  71.26436781609196


In [11]:
pool_data= pd.read_csv(r"C:\Users\shaha\Desktop\IBS\yelp\train_data4.csv",index_col=False)
train_data_text=clean(pool_data['text'])
test_data_text=clean(test_data['text'])
count_vect = CountVectorizer(stop_words='english')
transformer = TfidfTransformer(norm='l2',sublinear_tf=True)
x_train_counts = count_vect.fit_transform(train_data_text.apply(lambda x: ' '.join(x)))
x_train_tfidf = transformer.fit_transform(x_train_counts)
label=pool_data['label'].to_numpy()
model=fit_model(x_train_tfidf,label)
print("length of training data: ", len(pool_data))
print("length of testing data: ", len(test_data))
x_test_counts = count_vect.transform(test_data_text.apply(lambda x: ' '.join(x)))
x_test_tfidf = transformer.transform(x_test_counts)
predictions_train = model.predict(x_train_tfidf)
y_test=pool_data['label']
print('train accuracy : ',accuracy_score(y_test,predictions_train)*100)
predictions_test = model.predict(x_test_tfidf)
y_test=test_data['label']
print('test accuracy: ',accuracy_score(y_test,predictions_test)*100)



<ipython-input-3-592f685e97cd>:10: FutureWarning: The default value of regex will change from True to False in a future version.
  tex=tex.str.replace(r'\d+','')
<ipython-input-3-592f685e97cd>:12: FutureWarning: The default value of regex will change from True to False in a future version.
  tex = tex.str.replace("[^a-zA-Z#]", " " )
<ipython-input-3-592f685e97cd>:15: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  tex = tex.str.replace("$", " ")
c:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:541: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
c:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:545: RuntimeWarning: invalid value encountered in true_divide
  decisi

length of training data:  412
length of testing data:  87
train accuracy :  93.93203883495146
test accuracy:  79.3103448275862


In [30]:
pool_data= pd.read_csv(r"C:\Users\shaha\Desktop\IBS\yelp\train_data5.csv",index_col=False)
train_data_text=clean(pool_data['text'])
test_data_text=clean(test_data['text'])
count_vect = CountVectorizer(stop_words='english')
transformer = TfidfTransformer(norm='l2',sublinear_tf=True)
x_train_counts = count_vect.fit_transform(train_data_text.apply(lambda x: ' '.join(x)))
x_train_tfidf = transformer.fit_transform(x_train_counts)
label=pool_data['label'].to_numpy()
model=fit_model(x_train_tfidf,label)
print("length of training data: ", len(pool_data))
print("length of testing data: ", len(test_data))
x_test_counts = count_vect.transform(test_data_text.apply(lambda x: ' '.join(x)))
x_test_tfidf = transformer.transform(x_test_counts)
predictions_train = model.predict(x_train_tfidf)
y_test=pool_data['label']
print('train accuracy : ',accuracy_score(y_test,predictions_train)*100)
predictions_test = model.predict(x_test_tfidf)
y_test=test_data['label']
print('test accuracy: ',accuracy_score(y_test,predictions_test)*100)



<ipython-input-22-592f685e97cd>:10: FutureWarning: The default value of regex will change from True to False in a future version.
  tex=tex.str.replace(r'\d+','')
<ipython-input-22-592f685e97cd>:12: FutureWarning: The default value of regex will change from True to False in a future version.
  tex = tex.str.replace("[^a-zA-Z#]", " " )
<ipython-input-22-592f685e97cd>:15: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  tex = tex.str.replace("$", " ")
c:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:541: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
c:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:545: RuntimeWarning: invalid value encountered in true_divide
  dec

length of training data:  513
length of testing data:  87
train accuracy :  87.32943469785575
test accuracy:  75.86206896551724


In [31]:
pool_data= pd.read_csv(r"C:\Users\shaha\Desktop\IBS\yelp\train_data25.csv",index_col=False)
train_data_text=clean(pool_data['text'])
test_data_text=clean(test_data['text'])
count_vect = CountVectorizer(stop_words='english')
transformer = TfidfTransformer(norm='l2',sublinear_tf=True)
x_train_counts = count_vect.fit_transform(train_data_text.apply(lambda x: ' '.join(x)))
x_train_tfidf = transformer.fit_transform(x_train_counts)
label=pool_data['label'].to_numpy()
model=fit_model(x_train_tfidf,label)
print("length of training data: ", len(pool_data))
print("length of testing data: ", len(test_data))
x_test_counts = count_vect.transform(test_data_text.apply(lambda x: ' '.join(x)))
x_test_tfidf = transformer.transform(x_test_counts)
predictions_train = model.predict(x_train_tfidf)
y_test=pool_data['label']
print('train accuracy : ',accuracy_score(y_test,predictions_train)*100)
predictions_test = model.predict(x_test_tfidf)
y_test=test_data['label']
print('test accuracy: ',accuracy_score(y_test,predictions_test)*100)



<ipython-input-22-592f685e97cd>:10: FutureWarning: The default value of regex will change from True to False in a future version.
  tex=tex.str.replace(r'\d+','')
<ipython-input-22-592f685e97cd>:12: FutureWarning: The default value of regex will change from True to False in a future version.
  tex = tex.str.replace("[^a-zA-Z#]", " " )
<ipython-input-22-592f685e97cd>:15: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  tex = tex.str.replace("$", " ")
c:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:541: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
c:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:545: RuntimeWarning: invalid value encountered in true_divide
  dec

length of training data:  568
length of testing data:  87
train accuracy :  87.32394366197182
test accuracy:  79.3103448275862


In [32]:
pool_data= pd.read_csv(r"C:\Users\shaha\Desktop\IBS\yelp\train_data261.csv",index_col=False)
train_data_text=clean(pool_data['text'])
test_data_text=clean(test_data['text'])
count_vect = CountVectorizer(stop_words='english')
transformer = TfidfTransformer(norm='l2',sublinear_tf=True)
x_train_counts = count_vect.fit_transform(train_data_text.apply(lambda x: ' '.join(x)))
x_train_tfidf = transformer.fit_transform(x_train_counts)
label=pool_data['label'].to_numpy()
model=fit_model(x_train_tfidf,label)
print("length of training data: ", len(pool_data))
print("length of testing data: ", len(test_data))
x_test_counts = count_vect.transform(test_data_text.apply(lambda x: ' '.join(x)))
x_test_tfidf = transformer.transform(x_test_counts)
predictions_train = model.predict(x_train_tfidf)
y_test=pool_data['label']
print('train accuracy : ',accuracy_score(y_test,predictions_train)*100)
predictions_test = model.predict(x_test_tfidf)
y_test=test_data['label']
print('test accuracy: ',accuracy_score(y_test,predictions_test)*100)


<ipython-input-22-592f685e97cd>:10: FutureWarning: The default value of regex will change from True to False in a future version.
  tex=tex.str.replace(r'\d+','')
<ipython-input-22-592f685e97cd>:12: FutureWarning: The default value of regex will change from True to False in a future version.
  tex = tex.str.replace("[^a-zA-Z#]", " " )
<ipython-input-22-592f685e97cd>:15: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  tex = tex.str.replace("$", " ")
c:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:541: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
c:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:545: RuntimeWarning: invalid value encountered in true_divide
  dec

length of training data:  613
length of testing data:  87
train accuracy :  87.11256117455139
test accuracy:  80.45977011494253


In [33]:
pool_data= pd.read_csv(r"C:\Users\shaha\Desktop\IBS\yelp\train_data12.csv",index_col=False)
train_data_text=clean(pool_data['text'])
test_data_text=clean(test_data['text'])
count_vect = CountVectorizer(stop_words='english')
transformer = TfidfTransformer(norm='l2',sublinear_tf=True)
x_train_counts = count_vect.fit_transform(train_data_text.apply(lambda x: ' '.join(x)))
x_train_tfidf = transformer.fit_transform(x_train_counts)
label=pool_data['label'].to_numpy()
model=fit_model(x_train_tfidf,label)
print("length of training data: ", len(pool_data))
print("length of testing data: ", len(test_data))
x_test_counts = count_vect.transform(test_data_text.apply(lambda x: ' '.join(x)))
x_test_tfidf = transformer.transform(x_test_counts)
predictions_train = model.predict(x_train_tfidf)
y_test=pool_data['label']
print('train accuracy : ',accuracy_score(y_test,predictions_train)*100)
predictions_test = model.predict(x_test_tfidf)
y_test=test_data['label']
print('test accuracy: ',accuracy_score(y_test,predictions_test)*100)


<ipython-input-22-592f685e97cd>:10: FutureWarning: The default value of regex will change from True to False in a future version.
  tex=tex.str.replace(r'\d+','')
<ipython-input-22-592f685e97cd>:12: FutureWarning: The default value of regex will change from True to False in a future version.
  tex = tex.str.replace("[^a-zA-Z#]", " " )
<ipython-input-22-592f685e97cd>:15: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  tex = tex.str.replace("$", " ")
c:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:541: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
c:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:545: RuntimeWarning: invalid value encountered in true_divide
  dec

length of training data:  708
length of testing data:  87
train accuracy :  87.00564971751412
test accuracy:  77.01149425287356


In [34]:
pool_data= pd.read_csv(r"C:\Users\shaha\Desktop\IBS\yelp\train_data113.csv",index_col=False)
train_data_text=clean(pool_data['text'])
test_data_text=clean(test_data['text'])
count_vect = CountVectorizer(stop_words='english')
transformer = TfidfTransformer(norm='l2',sublinear_tf=True)
x_train_counts = count_vect.fit_transform(train_data_text.apply(lambda x: ' '.join(x)))
x_train_tfidf = transformer.fit_transform(x_train_counts)
label=pool_data['label'].to_numpy()
model=fit_model(x_train_tfidf,label)
print("length of training data: ", len(pool_data))
print("length of testing data: ", len(test_data))
x_test_counts = count_vect.transform(test_data_text.apply(lambda x: ' '.join(x)))
x_test_tfidf = transformer.transform(x_test_counts)
predictions_train = model.predict(x_train_tfidf)
y_test=pool_data['label']
print('train accuracy : ',accuracy_score(y_test,predictions_train)*100)
predictions_test = model.predict(x_test_tfidf)
y_test=test_data['label']
print('test accuracy: ',accuracy_score(y_test,predictions_test)*100)



<ipython-input-22-592f685e97cd>:10: FutureWarning: The default value of regex will change from True to False in a future version.
  tex=tex.str.replace(r'\d+','')
<ipython-input-22-592f685e97cd>:12: FutureWarning: The default value of regex will change from True to False in a future version.
  tex = tex.str.replace("[^a-zA-Z#]", " " )
<ipython-input-22-592f685e97cd>:15: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  tex = tex.str.replace("$", " ")
c:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:541: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
c:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:545: RuntimeWarning: invalid value encountered in true_divide
  dec

length of training data:  807
length of testing data:  87
train accuracy :  90.21065675340768
test accuracy:  75.86206896551724


In [35]:
pool_data= pd.read_csv(r"C:\Users\shaha\Desktop\IBS\yelp\train_data114.csv",index_col=False)
train_data_text=clean(pool_data['text'])
test_data_text=clean(test_data['text'])
count_vect = CountVectorizer(stop_words='english')
transformer = TfidfTransformer(norm='l2',sublinear_tf=True)
x_train_counts = count_vect.fit_transform(train_data_text.apply(lambda x: ' '.join(x)))
x_train_tfidf = transformer.fit_transform(x_train_counts)
label=pool_data['label'].to_numpy()
model=fit_model(x_train_tfidf,label)
print("length of training data: ", len(pool_data))
print("length of testing data: ", len(test_data))
x_test_counts = count_vect.transform(test_data_text.apply(lambda x: ' '.join(x)))
x_test_tfidf = transformer.transform(x_test_counts)
predictions_train = model.predict(x_train_tfidf)
y_test=pool_data['label']
print('train accuracy : ',accuracy_score(y_test,predictions_train)*100)
predictions_test = model.predict(x_test_tfidf)
y_test=test_data['label']
print('test accuracy: ',accuracy_score(y_test,predictions_test)*100)



<ipython-input-22-592f685e97cd>:10: FutureWarning: The default value of regex will change from True to False in a future version.
  tex=tex.str.replace(r'\d+','')
<ipython-input-22-592f685e97cd>:12: FutureWarning: The default value of regex will change from True to False in a future version.
  tex = tex.str.replace("[^a-zA-Z#]", " " )
<ipython-input-22-592f685e97cd>:15: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  tex = tex.str.replace("$", " ")
c:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:541: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
c:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:545: RuntimeWarning: invalid value encountered in true_divide
  dec

length of training data:  905
length of testing data:  87
train accuracy :  90.49723756906077
test accuracy:  82.75862068965517


In [36]:
pool_data= pd.read_csv(r"C:\Users\shaha\Desktop\IBS\yelp\train_data115.csv",index_col=False)
train_data_text=clean(pool_data['text'])
test_data_text=clean(test_data['text'])
count_vect = CountVectorizer(stop_words='english')
transformer = TfidfTransformer(norm='l2',sublinear_tf=True)
x_train_counts = count_vect.fit_transform(train_data_text.apply(lambda x: ' '.join(x)))
x_train_tfidf = transformer.fit_transform(x_train_counts)
label=pool_data['label'].to_numpy()
model=fit_model(x_train_tfidf,label)
print("length of training data: ", len(pool_data))
print("length of testing data: ", len(test_data))
x_test_counts = count_vect.transform(test_data_text.apply(lambda x: ' '.join(x)))
x_test_tfidf = transformer.transform(x_test_counts)
predictions_train = model.predict(x_train_tfidf)
y_test=pool_data['label']
print('train accuracy : ',accuracy_score(y_test,predictions_train)*100)
predictions_test = model.predict(x_test_tfidf)
y_test=test_data['label']
print('test accuracy: ',accuracy_score(y_test,predictions_test)*100)



<ipython-input-22-592f685e97cd>:10: FutureWarning: The default value of regex will change from True to False in a future version.
  tex=tex.str.replace(r'\d+','')
<ipython-input-22-592f685e97cd>:12: FutureWarning: The default value of regex will change from True to False in a future version.
  tex = tex.str.replace("[^a-zA-Z#]", " " )
<ipython-input-22-592f685e97cd>:15: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  tex = tex.str.replace("$", " ")
c:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:541: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
c:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:545: RuntimeWarning: invalid value encountered in true_divide
  dec

length of training data:  1001
length of testing data:  87
train accuracy :  89.41058941058941
test accuracy:  83.9080459770115


In [37]:
pool_data= pd.read_csv(r"C:\Users\shaha\Desktop\IBS\yelp\train_data116.csv",index_col=False)
train_data_text=clean(pool_data['text'])
test_data_text=clean(test_data['text'])
count_vect = CountVectorizer(stop_words='english')
transformer = TfidfTransformer(norm='l2',sublinear_tf=True)
x_train_counts = count_vect.fit_transform(train_data_text.apply(lambda x: ' '.join(x)))
x_train_tfidf = transformer.fit_transform(x_train_counts)
label=pool_data['label'].to_numpy()
model=fit_model(x_train_tfidf,label)
print("length of training data: ", len(pool_data))
print("length of testing data: ", len(test_data))
x_test_counts = count_vect.transform(test_data_text.apply(lambda x: ' '.join(x)))
x_test_tfidf = transformer.transform(x_test_counts)
predictions_train = model.predict(x_train_tfidf)
y_test=pool_data['label']
print('train accuracy : ',accuracy_score(y_test,predictions_train)*100)
predictions_test = model.predict(x_test_tfidf)
y_test=test_data['label']
print('test accuracy: ',accuracy_score(y_test,predictions_test)*100)


<ipython-input-22-592f685e97cd>:10: FutureWarning: The default value of regex will change from True to False in a future version.
  tex=tex.str.replace(r'\d+','')
<ipython-input-22-592f685e97cd>:12: FutureWarning: The default value of regex will change from True to False in a future version.
  tex = tex.str.replace("[^a-zA-Z#]", " " )
<ipython-input-22-592f685e97cd>:15: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  tex = tex.str.replace("$", " ")
c:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:541: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
c:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:545: RuntimeWarning: invalid value encountered in true_divide
  dec

length of training data:  1098
length of testing data:  87
train accuracy :  88.88888888888889
test accuracy:  79.3103448275862


In [38]:
pool_data= pd.read_csv(r"C:\Users\shaha\Desktop\IBS\yelp\train_data117.csv",index_col=False)
train_data_text=clean(pool_data['text'])
test_data_text=clean(test_data['text'])
count_vect = CountVectorizer(stop_words='english')
transformer = TfidfTransformer(norm='l2',sublinear_tf=True)
x_train_counts = count_vect.fit_transform(train_data_text.apply(lambda x: ' '.join(x)))
x_train_tfidf = transformer.fit_transform(x_train_counts)
label=pool_data['label'].to_numpy()
model=fit_model(x_train_tfidf,label)
print("length of training data: ", len(pool_data))
print("length of testing data: ", len(test_data))
x_test_counts = count_vect.transform(test_data_text.apply(lambda x: ' '.join(x)))
x_test_tfidf = transformer.transform(x_test_counts)
predictions_train = model.predict(x_train_tfidf)
y_test=pool_data['label']
print('train accuracy : ',accuracy_score(y_test,predictions_train)*100)
predictions_test = model.predict(x_test_tfidf)
y_test=test_data['label']
print('test accuracy: ',accuracy_score(y_test,predictions_test)*100)



<ipython-input-22-592f685e97cd>:10: FutureWarning: The default value of regex will change from True to False in a future version.
  tex=tex.str.replace(r'\d+','')
<ipython-input-22-592f685e97cd>:12: FutureWarning: The default value of regex will change from True to False in a future version.
  tex = tex.str.replace("[^a-zA-Z#]", " " )
<ipython-input-22-592f685e97cd>:15: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  tex = tex.str.replace("$", " ")
c:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:541: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
c:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:545: RuntimeWarning: invalid value encountered in true_divide
  dec

length of training data:  1197
length of testing data:  87
train accuracy :  87.88638262322472
test accuracy:  72.41379310344827


In [39]:
pool_data= pd.read_csv(r"C:\Users\shaha\Desktop\IBS\yelp\train_data118.csv",index_col=False)
train_data_text=clean(pool_data['text'])
test_data_text=clean(test_data['text'])
count_vect = CountVectorizer(stop_words='english')
transformer = TfidfTransformer(norm='l2',sublinear_tf=True)
x_train_counts = count_vect.fit_transform(train_data_text.apply(lambda x: ' '.join(x)))
x_train_tfidf = transformer.fit_transform(x_train_counts)
label=pool_data['label'].to_numpy()
model=fit_model(x_train_tfidf,label)
print("length of training data: ", len(pool_data))
print("length of testing data: ", len(test_data))
x_test_counts = count_vect.transform(test_data_text.apply(lambda x: ' '.join(x)))
x_test_tfidf = transformer.transform(x_test_counts)
predictions_train = model.predict(x_train_tfidf)
y_test=pool_data['label']
print('train accuracy : ',accuracy_score(y_test,predictions_train)*100)
predictions_test = model.predict(x_test_tfidf)
y_test=test_data['label']
print('test accuracy: ',accuracy_score(y_test,predictions_test)*100)



<ipython-input-22-592f685e97cd>:10: FutureWarning: The default value of regex will change from True to False in a future version.
  tex=tex.str.replace(r'\d+','')
<ipython-input-22-592f685e97cd>:12: FutureWarning: The default value of regex will change from True to False in a future version.
  tex = tex.str.replace("[^a-zA-Z#]", " " )
<ipython-input-22-592f685e97cd>:15: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  tex = tex.str.replace("$", " ")
c:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:541: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
c:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:545: RuntimeWarning: invalid value encountered in true_divide
  dec

length of training data:  1296
length of testing data:  87
train accuracy :  86.65123456790124
test accuracy:  81.60919540229885


In [40]:
pool_data= pd.read_csv(r"C:\Users\shaha\Desktop\IBS\yelp\train_data119.csv",index_col=False)
train_data_text=clean(pool_data['text'])
test_data_text=clean(test_data['text'])
count_vect = CountVectorizer(stop_words='english')
transformer = TfidfTransformer(norm='l2',sublinear_tf=True)
x_train_counts = count_vect.fit_transform(train_data_text.apply(lambda x: ' '.join(x)))
x_train_tfidf = transformer.fit_transform(x_train_counts)
label=pool_data['label'].to_numpy()
model=fit_model(x_train_tfidf,label)
print("length of training data: ", len(pool_data))
print("length of testing data: ", len(test_data))
x_test_counts = count_vect.transform(test_data_text.apply(lambda x: ' '.join(x)))
x_test_tfidf = transformer.transform(x_test_counts)
predictions_train = model.predict(x_train_tfidf)
y_test=pool_data['label']
print('train accuracy : ',accuracy_score(y_test,predictions_train)*100)
predictions_test = model.predict(x_test_tfidf)
y_test=test_data['label']
print('test accuracy: ',accuracy_score(y_test,predictions_test)*100)



<ipython-input-22-592f685e97cd>:10: FutureWarning: The default value of regex will change from True to False in a future version.
  tex=tex.str.replace(r'\d+','')
<ipython-input-22-592f685e97cd>:12: FutureWarning: The default value of regex will change from True to False in a future version.
  tex = tex.str.replace("[^a-zA-Z#]", " " )
<ipython-input-22-592f685e97cd>:15: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  tex = tex.str.replace("$", " ")
c:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:541: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
c:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:545: RuntimeWarning: invalid value encountered in true_divide
  dec

length of training data:  1394
length of testing data:  87
train accuracy :  84.50502152080345
test accuracy:  80.45977011494253


In [41]:
pool_data= pd.read_csv(r"C:\Users\shaha\Desktop\IBS\yelp\train_data120.csv",index_col=False)
train_data_text=clean(pool_data['text'])
test_data_text=clean(test_data['text'])
count_vect = CountVectorizer(stop_words='english')
transformer = TfidfTransformer(norm='l2',sublinear_tf=True)
x_train_counts = count_vect.fit_transform(train_data_text.apply(lambda x: ' '.join(x)))
x_train_tfidf = transformer.fit_transform(x_train_counts)
label=pool_data['label'].to_numpy()
model=fit_model(x_train_tfidf,label)
print("length of training data: ", len(pool_data))
print("length of testing data: ", len(test_data))
x_test_counts = count_vect.transform(test_data_text.apply(lambda x: ' '.join(x)))
x_test_tfidf = transformer.transform(x_test_counts)
predictions_train = model.predict(x_train_tfidf)
y_test=pool_data['label']
print('train accuracy : ',accuracy_score(y_test,predictions_train)*100)
predictions_test = model.predict(x_test_tfidf)
y_test=test_data['label']
print('test accuracy: ',accuracy_score(y_test,predictions_test)*100)



<ipython-input-22-592f685e97cd>:10: FutureWarning: The default value of regex will change from True to False in a future version.
  tex=tex.str.replace(r'\d+','')
<ipython-input-22-592f685e97cd>:12: FutureWarning: The default value of regex will change from True to False in a future version.
  tex = tex.str.replace("[^a-zA-Z#]", " " )
<ipython-input-22-592f685e97cd>:15: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  tex = tex.str.replace("$", " ")
c:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:541: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
c:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:545: RuntimeWarning: invalid value encountered in true_divide
  dec

length of training data:  1499
length of testing data:  87
train accuracy :  84.58972648432288
test accuracy:  83.9080459770115


In [42]:
pool_data= pd.read_csv(r"C:\Users\shaha\Desktop\IBS\yelp\train_data121.csv",index_col=False)
train_data_text=clean(pool_data['text'])
test_data_text=clean(test_data['text'])
count_vect = CountVectorizer(stop_words='english')
transformer = TfidfTransformer(norm='l2',sublinear_tf=True)
x_train_counts = count_vect.fit_transform(train_data_text.apply(lambda x: ' '.join(x)))
x_train_tfidf = transformer.fit_transform(x_train_counts)
label=pool_data['label'].to_numpy()
model=fit_model(x_train_tfidf,label)
print("length of training data: ", len(pool_data))
print("length of testing data: ", len(test_data))
x_test_counts = count_vect.transform(test_data_text.apply(lambda x: ' '.join(x)))
x_test_tfidf = transformer.transform(x_test_counts)
predictions_train = model.predict(x_train_tfidf)
y_test=pool_data['label']
print('train accuracy : ',accuracy_score(y_test,predictions_train)*100)
predictions_test = model.predict(x_test_tfidf)
y_test=test_data['label']
print('test accuracy: ',accuracy_score(y_test,predictions_test)*100)



<ipython-input-22-592f685e97cd>:10: FutureWarning: The default value of regex will change from True to False in a future version.
  tex=tex.str.replace(r'\d+','')
<ipython-input-22-592f685e97cd>:12: FutureWarning: The default value of regex will change from True to False in a future version.
  tex = tex.str.replace("[^a-zA-Z#]", " " )
<ipython-input-22-592f685e97cd>:15: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  tex = tex.str.replace("$", " ")
c:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:541: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
c:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:545: RuntimeWarning: invalid value encountered in true_divide
  dec

length of training data:  1600
length of testing data:  87
train accuracy :  86.125
test accuracy:  80.45977011494253


In [24]:
#This is used to clean the unlabelld pool of reviews
clean_text_pool=clean(rev['text'])

<ipython-input-3-592f685e97cd>:10: FutureWarning: The default value of regex will change from True to False in a future version.
  tex=tex.str.replace(r'\d+','')
<ipython-input-3-592f685e97cd>:12: FutureWarning: The default value of regex will change from True to False in a future version.
  tex = tex.str.replace("[^a-zA-Z#]", " " )
<ipython-input-3-592f685e97cd>:15: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  tex = tex.str.replace("$", " ")


In [25]:
#This will help to create a new csv file in which we will have labelled text and queried text (which need to be labelled)
x_counts = count_vect.transform(clean_text_pool.apply(lambda x: ' '.join(x)))
x_tfidf = transformer.transform(x_counts)
sample=model.predict_proba(x_tfidf)
rev_id=entropy_sampling(sample)
y=[]
train_ai=rev.loc[rev_id]
train_ai.to_csv(r"C:\Users\shaha\Desktop\IBS\yelp\train_data121.csv", mode='a', index=False, header=False)